In [1]:
import numpy as np
import pickle

# read pickel file
with open('run/sim_results/res_sum_all.pkl', 'rb') as f:
    data = pickle.load(f)

In [2]:
# present coverage
for min_degree, max_degree in [(1,2), (1,4), (1,8), (1,10),
                               (2,5), (2,6), (2,8), (2,10),
                               (3,10), (4,10)]:
    
    coverage = data[(min_degree, max_degree)][f'model[dr_em]_samplesize[800]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[None]]']['coverage_rate']
    print(f"({min_degree}, {max_degree}): ")
    print(coverage[:4])

(1, 2): 
[0.9075 0.9205 0.9415 0.9475]
(1, 4): 
[0.966  0.959  0.941  0.9305]
(1, 8): 
[0.9715 0.96   0.9565 0.942 ]
(1, 10): 
[0.9745 0.9675 0.948  0.932 ]
(2, 5): 
[0.958  0.9495 0.939  0.927 ]
(2, 6): 
[0.97   0.9655 0.9295 0.9225]
(2, 8): 
[0.973  0.964  0.9425 0.9195]
(2, 10): 
[0.9815 0.973  0.9345 0.904 ]
(3, 10): 
[0.991 0.982 0.944 0.807]
(4, 10): 
[0.994  0.9935 0.9395 0.464 ]


In [3]:
# try different h
import numpy as np
from utils import *
from utils import *
from dgp import sample_network_chain, get_graph, sample_Y1, agcEffect
import numpy as np


def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,-2*min_degree,2,2,0.1,-1,0.1,2,0.1,0])   # shape (10,)

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, None, None, None, ground_truth1

num_sample = 300
num_burn = 20
n_cpu = 10

def get_coverage(min_degree, max_degree, sample_size, h):
    adj1, _, _, _, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

    model = 'dr_em'
    mispec = None

    print('Processing model:', model, 'with mispecification:', mispec, f'min_max_{min_degree}_{max_degree}')
    results1 = np.load(f'run/sim_results/sim_{model}_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_{mispec}.npy', allow_pickle=True)
    results2 = np.load(f'run/sim_results/sim_{model}_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_{mispec}.npy', allow_pickle=True)

    res_ = compute_stats(results1, results2, ground_truth1, adj1, h)
    coverage = res_['coverage_rate']
            
    return coverage

In [ ]:
for min_degree, max_degree in [(1,2), (1,4), (1,8), (1,10),
                                 (2,5), (2,6), (2,8), (2,10),
                                 (3,10), (4,10)]:
     sample_size = 800
     res_sum = get_coverage(min_degree, max_degree, sample_size, 3)
     print(res_sum)

1 2
Processing model: dr_em with mispecification: None min_max_1_2


 23%|██▎       | 461/2000 [00:16<00:57, 26.80it/s]

In [ ]:
# present mse

for min_degree, max_degree in [(1,2), (2,5), (3,10)]:
    for mispec in ['outcome']: #[None, 'outcome', 'treatment']:
        for model in ['dr_em', 'ag']:
            if mispec == 'treatment' and model == 'ag':
                mispec = None
            bias = data[(min_degree, max_degree)][f'model[{model}]_samplesize[800]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]']['bias']
            mse = data[(min_degree, max_degree)][f'model[{model}]_samplesize[800]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]']['mse']
            if model == 'dr_em':
                true_value = data[(min_degree, max_degree)][f'model[{model}]_samplesize[800]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]']['true_effect']
            else:
                true_value = data[(min_degree, max_degree)][f'model[{model}]_samplesize[800]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]']['ground_truth']
            print(f"({min_degree}, {max_degree}, {mispec}, {model}): ")
            print("True effect:", true_value[:4])
            print("Bias:", bias[:4])
            print("RMSE:", np.sqrt(mse[:4]))


(1, 2, outcome, dr_em): 
True effect: [ 0.7017625 -0.2298375  0.1784     0.3535625]
Bias: [0.01081746 0.01605687 0.00879704 0.01324783]
RMSE: [0.051674   0.07783484 0.03766411 0.0806457 ]
(1, 2, outcome, ag): 
True effect: [ 0.7017625 -0.2298375  0.1784     0.3535625]
Bias: [0.08701326 0.18736335 0.0743367  0.09265836]
RMSE: [0.08857518 0.18931448 0.07846876 0.1011044 ]
(2, 5, outcome, dr_em): 
True effect: [ 0.7027375 -0.3108     0.2020375  0.5181875]
Bias: [-0.0014591  -0.00532166  0.01995111  0.06994423]
RMSE: [0.03809644 0.05797995 0.05909992 0.20982901]
(2, 5, outcome, ag): 
True effect: [ 0.7027375 -0.3108     0.2020375  0.5181875]
Bias: [0.05736601 0.12139075 0.20343849 0.16366608]
RMSE: [0.05883391 0.12334823 0.20458926 0.16887755]
(3, 10, outcome, dr_em): 
True effect: [ 0.648875  -0.4016875  0.2073625  0.618325 ]
Bias: [-0.00504453 -0.01943272  0.03542031  0.14936402]
RMSE: [0.02748872 0.04764727 0.09895893 0.27443196]
(3, 10, outcome, ag): 
True effect: [ 0.648875  -0.401687